In [1]:
import numpy as np
import matplotlib.pyplot as plt

import polyseq as pseq

%matplotlib inline
numpy.random.seed(0)

NameError: name 'numpy' is not defined

In [ ]:
pseq.io.download_example_data()

data = pseq.io.load_example()
data.head()

In [ ]:
dropped = data.drop_genes(num_cells=1)
print(data.shape)
print(dropped.shape)

In [ ]:
stats = pseq.summary.summarize(dropped)
stats

In [ ]:
normed = dropped.log_normalize()
total = normed.sum(axis=1)
regressed = pseq.regress(normed, total[:, np.newaxis])
regressed.head()

In [ ]:
reduced = pseq.dim.pca(regressed, k=15)

plt.scatter(reduced['pc-0'], reduced['pc-1'], s=10)
reduced.head()

In [ ]:
clusters = pseq.clustering.graph_cluster(reduced, n_neighbors=100)
print("# of clusters: ", len(np.unique(clusters)))

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=10.0, verbose=True).fit_transform(reduced)
plt.scatter(tsne[:, 0], tsne[:, 1], s=10, c=clusters, cmap='tab20')

In [ ]:
top_genes = pseq.upregulated(normed, clusters, n=3)

print("\n\n---\n")
for i, genes in enumerate(top_genes):
    print(genes)

In [ ]:
genes = sum(top_genes, []) # append all lists into single list
pseq.viz.violins(normed, genes, groups=clusters, cluster_genes=False)